# data loading and initial filtering

In [1]:
import math
from data_loader import Data_loader
from pair_filter import NPD_filter
from pair_filter import OLS_filter
import matplotlib.pyplot as plt



# loading data
start_idx = 0


dl = Data_loader(num_month_train=8, num_month_test=2)
window_train, window_test = dl.get_window(start_idx=start_idx)



# NPD selection
NPD = NPD_filter(window_train)
pairs, distance = NPD.select_pairs(percentile=5, verbose=True)


35432 pairs selected
min distance 0.0
max distance 0.7758473415183631


In [2]:
# OLS selection
OLS = OLS_filter(window_train)
pairs, params = OLS.select_pairs(pairs, verbose=True, confidence_level=0.05)

12144 out of 35432 pairs passed the test
pass rate: 0.34


# select top-k pairs accorindg to train sharpe ratio

In [4]:
from backtest import *

top_k = 50

bt = PairTradingBacktester_TrainTest(pairs[:300], params[:300], window_train, window_test) # for testing purpose
# bt = PairTradingBacktester_TrainTest(pairs, params, window_train, window_test) # complete
summary = bt.run(top_k=top_k)


spreads_train  = bt.train_spreads_df
spreads_test  = bt.test_spreads_df
returns_test   = bt.test_returns_df
returns_train   = bt.train_returns_df
comp_weights=bt.comp_weights

Train back-test:   0%|          | 0/300 [00:00<?, ?it/s]

Test back-test:   0%|          | 0/50 [00:00<?, ?it/s]

# EW optimizer

In [5]:
from optimizers import EW_optimizer

optimal_weights=EW_optimizer(returns_train) # train on training data

port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data


port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

(CVXPY) May 09 05:59:43 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib\n  Referenced from: <E25E40AB-7857-39B9-8DE7-28B7B0E4806B> /Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so\n  Reason: tried: '/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)")
(CVXPY) May 09 05:59:43 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlope

4.154880494909343

# Sharpe Optimizer

In [6]:
from optimizers import Sharpe_optimizer

optimal_weights=Sharpe_optimizer(returns_train)  # train on training data
port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data
port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

4.669804922543596

In [7]:
port_ret

date
1990-07-02    0.000000
1990-07-03    0.003723
1990-07-05   -0.002052
1990-07-06    0.003221
1990-07-09    0.012017
1990-07-10    0.003471
1990-07-11    0.012385
1990-07-12   -0.002068
1990-07-13    0.018666
1990-07-16   -0.000239
1990-07-17    0.005135
1990-07-18    0.001995
1990-07-19    0.005914
1990-07-20    0.006273
1990-07-23    0.012112
1990-07-24    0.002762
1990-07-25    0.372964
1990-07-26    0.004834
1990-07-27   -0.001771
1990-07-30    0.019225
1990-07-31    0.017632
dtype: float64